 
  <h1><center><font size=10>Google Query</center></font></h1>
  <h1><center>Exploratory Data Analysis - Google Query Case Study</center></h1>

<p>This data analysis delves into Google search data by DMA. By examining this data, we aim to uncover insights into:
</p>
<ol>
    <li>
        <p>Searches for NBA team-related terms.</p>
    </li>
    <li>
        <p>The relationship between NBA teams' records (winning percentage) and searches for the teams.</p>
    </li>
    
</ol>

In [1]:
# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd
import requests
from pathlib import Path

# Libraries to help with data visualization
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [2]:
# Study data files
google_query_path = "data/Google_Bigquery_Top_Search_Week_Ending_2024-06-02.csv"
google_query_top_search_path = "data/Google_Bigquery_Top_Search_Jan_Jun_2024.csv"

google_query_data = pd.read_csv(google_query_path)
google_query_top_search_data = pd.read_csv(google_query_top_search_path)

# Read the CSV into a Pandas DataFrame
google_queryDF = pd.DataFrame(google_query_data)

# Display the data table for preview
google_queryDF.head()

,week,score,rank,refresh_date,dma_name,dma_id,term
0,2024-05-26,100.0,13,2024-05-30,Charlotte NC,517,All Eyes on Rafah meaning
1,2024-05-26,100.0,13,2024-05-30,Baltimore MD,512,All Eyes on Rafah meaning
2,2024-05-26,100.0,13,2024-05-30,Grand Rapids-Kalamazoo-Battle Creek MI,563,All Eyes on Rafah meaning
3,2024-05-26,100.0,13,2024-05-30,Philadelphia PA,504,All Eyes on Rafah meaning
4,2024-05-26,100.0,13,2024-05-30,Ft. Myers-Naples FL,571,All Eyes on Rafah meaning


In [4]:
# Print the last five rows of data to the screen
google_queryDF.tail()

,week,score,rank,refresh_date,dma_name,dma_id,term
98905,2024-05-26,NaN,12,2024-06-03,Presque Isle ME,552,Maldives
98906,2024-05-26,14.0,12,2024-06-03,Corpus Christi TX,600,Maldives
98907,2024-06-02,21.0,12,2024-06-03,Corpus Christi TX,600,Maldives
98908,2024-05-26,NaN,12,2024-06-03,Jackson TN,639,Maldives
98909,2024-06-02,NaN,12,2024-06-03,Great Falls MT,755,Maldives


In [7]:
# Convert 'week' column to datetime format
google_queryDF['week'] = pd.to_datetime(google_queryDF['week'])

google_queryDF

# Group by month, term. and DMA and aggregate the data
monthly_data = google_queryDF.groupby(['term', 'dma_name', google_queryDF['week'].dt.to_period('M')]).agg({
    'score': 'mean',
    'rank': 'mean',
    'refresh_date': 'max',  # Get the latest refresh_date in the month
    'dma_id': 'first',       # Get the first dma_id in the month
  }).reset_index()

monthly_data.rename(columns={'week': 'month'}, inplace=True)

# Display the monthly data
print(monthly_data)
monthly_data.tail()

#Print out a sample case (searches for "Celtics" in SLC DMA) of aggregating search terms by month 
filtered_df = google_queryDF[(google_queryDF['term'] == 'Celtics') & (google_queryDF['dma_name'] == 'Salt Lake City UT')]
print(filtered_df)

month_filtered_df = monthly_data[(monthly_data['term'] == 'Celtics') & (monthly_data['dma_name'] == 'Salt Lake City UT')]
month_filtered_df

           term                            dma_name    month  score  rank  \
0      ABC News               Abilene-Sweetwater TX  2024-05    8.0  16.0   
1      ABC News                           Albany GA  2024-05    NaN  16.0   
2      ABC News          Albany-Schenectady-Troy NY  2024-05   13.0  16.0   
3      ABC News             Albuquerque-Santa Fe NM  2024-05   10.0  16.0   
4      ABC News                       Alexandria LA  2024-05  100.0  16.0   
...         ...                                 ...      ...    ...   ...   
65515    Wolves                       Wilmington NC  2024-05   80.0  10.0   
65516    Wolves  Yakima-Pasco-Richland-Kennewick WA  2024-05   75.0  10.0   
65517    Wolves                       Youngstown OH  2024-05   82.0  10.0   
65518    Wolves                Yuma AZ-El Centro CA  2024-05   37.0  10.0   
65519    Wolves                       Zanesville OH  2024-05    9.0  10.0   

      refresh_date  dma_id  
0       2024-06-01     662  
1       2024-06-0

,term,dma_name,month,score,rank,refresh_date,dma_id
5998,Celtics,Salt Lake City UT,2024-05,50.8,16.4,2024-06-07,770
5999,Celtics,Salt Lake City UT,2024-06,37.0,1.0,2024-06-07,770


In [77]:
# Import the data file containing NBA teams' winning percentages, by month
NBA_team_path = "data/NBA 23-24 Records by Team, by Month - Merge Candidate.csv"

NBA_team_data = pd.read_csv(NBA_team_path)

# Read the CSV into a Pandas DataFrame
NBA_recordsDF = pd.DataFrame(NBA_team_data)

# Display the data table for preview
NBA_recordsDF.head(30)

# Unpivot the data to create row for each month for each team, 210 (= 30 teams x 7 months) rows total
unpivoted_data = NBA_recordsDF.melt(id_vars='Team', var_name='month', value_name='value')
pivoted_data=unpivoted_data.sort_values(by=["Team", "month"],ascending=True)

# Display the unpivoted data
print(pivoted_data)

                   Team    month  value
20        Atlanta Hawks  2023-11  0.500
50        Atlanta Hawks  2023-12  0.500
80        Atlanta Hawks  2024-01  0.406
110       Atlanta Hawks  2024-02  0.426
140       Atlanta Hawks  2024-03  0.441
..                  ...      ...    ...
88   Washington Wizards  2024-01  0.188
118  Washington Wizards  2024-02  0.191
148  Washington Wizards  2024-03  0.153
178  Washington Wizards  2024-04  0.187
208  Washington Wizards  2024-05  0.183

[210 rows x 3 columns]


In [32]:
# Calculate the total count per DMA
dma_counts_df = google_queryDF.groupby('dma_name')['dma_name'].count()

# Display results
dma_counts_df

# Calculate the total count per weej
week_counts_df = google_queryDF.groupby('week')['week'].count()

# Display results
week_counts_df

# Calculate the total count per rank
rank_counts_df = google_queryDF.groupby('rank')['rank'].count()

# Display results
rank_counts_df

# Calculate the total count per rank
filtered_df = google_queryDF[google_queryDF['rank'].isin([1,2,3,4,5])]
term_counts = filtered_df['term'].value_counts()

#value_counts().reset_index().rename(columns={'index': 'term'}) 
top_ranks_df = term_counts[term_counts == term_counts.max()]
#value_counts().reset_index().rename(columns={'index': 'term'}) 

# Display results
top_5s_df=filtered_df[filtered_df['term'].isin(top_ranks_df.index)]

top_5s_df.head(20)


,week,score,rank,refresh_date,dma_name,dma_id,term
4954,2024-05-26,67.0,2,2024-06-01,Portland-Auburn ME,500,Real Madrid
4955,2024-05-26,80.0,2,2024-06-01,Charlotte NC,517,Real Madrid
4956,2024-05-26,67.0,2,2024-06-01,Greenville-New Bern-Washington NC,545,Real Madrid
4957,2024-05-26,56.0,2,2024-06-01,Wilmington NC,550,Real Madrid
4958,2024-05-26,76.0,2,2024-06-01,Chattanooga TN,575,Real Madrid
4959,2024-05-26,85.0,2,2024-06-01,Baltimore MD,512,Real Madrid
4960,2024-05-26,100.0,2,2024-06-01,Marquette MI,553,Real Madrid
4961,2024-05-26,89.0,2,2024-06-01,Grand Rapids-Kalamazoo-Battle Creek MI,563,Real Madrid
4962,2024-05-26,76.0,2,2024-06-01,Wilkes Barre-Scranton PA,577,Real Madrid
4963,2024-05-26,24.0,2,2024-06-01,Rapid City SD,764,Real Madrid
